In [1]:
# Reference
# https://yurimkoo.github.io/python/2019/09/14/connect-db-with-python.html

import pandas as pd
import requests
import pymysql
import json
import os
from dotenv import load_dotenv
from sqlalchemy import create_engine
from chicken_dinner.pubgapi import PUBG
from functions.pubgapi import get_tournament_info
from functions.pubgapi import get_match_info
from functions.pubgapi import get_match_info2
from functions.pubgapi import get_match_participant

In [2]:
# Load api key & RDS info (endpoint="rds 엔드포인트", dbname="db 이름", username="마스터 사용자 이름", password="rds 비밀번호")
load_dotenv()
api_key = os.environ.get("API_KEY")
endpoint = os.environ.get("AWS_endpoint")
dbname = os.environ.get('AWS_dbname')
username = os.environ.get("AWS_username")
password = os.environ.get("AWS_password")

# PUBG class를 토너먼트용 class로 custom
PUBG_prime = PUBG(api_key=api_key, shard='pc-tournament', gzip=True)

In [3]:
# Tournament id 값 조회
tournament_info = get_tournament_info(api_key)
tournament_id = tournament_info["id"]

print(f"{len(tournament_id)} 개의 tournament_id 조회 완료")

# Match id 값 조회

# tournament_index를 지정하여 id값 출력
idx = 0
tournament_index_id = tournament_id[idx]

cur_match_info = get_match_info(api_key, tournament_index_id)
num_match = len(cur_match_info["matchId"])
print(f"Tournament name: {tournament_index_id} ({num_match} matches)")

# Match participant 조회
match_info_id = cur_match_info["matchId"]
match_participant = get_match_participant(api_key, match_info_id)
match_participant

466 개의 tournament_id 조회 완료
Tournament name: eu-trt22 (9 matches)


,match_id,player_id,team_rank,dbnos,assists,boosts,damage_dealt,headshot_kills,heals,kill_streaks,kills,longest_kill,revives,team_kills,time_survived
0,fd8981d9-61ea-4bf7-9511-5c58eba7010b,Goop,2,0,0,0,0.00000,0,0,0,0,0.000000,0,0,1407
1,fd8981d9-61ea-4bf7-9511-5c58eba7010b,MarcWithaC,4,0,0,0,0.00000,0,0,0,0,0.000000,0,0,828
2,fd8981d9-61ea-4bf7-9511-5c58eba7010b,Jamie,5,0,0,0,0.00000,0,0,0,0,0.000000,0,0,285
3,fd8981d9-61ea-4bf7-9511-5c58eba7010b,Gus79,3,0,0,0,0.00000,0,0,0,0,0.000000,0,0,1274
4,fd8981d9-61ea-4bf7-9511-5c58eba7010b,BetaFoz,1,0,0,0,0.00000,0,0,0,0,0.000000,0,0,1410
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
327,bfb2c8d8-7733-4f99-b236-a980418ac0c9,WTN_Sinco,1,1,0,2,45.91400,0,2,0,0,0.000000,0,0,1161
328,bfb2c8d8-7733-4f99-b236-a980418ac0c9,10K_xLyron,1,3,0,2,265.09628,0,3,1,3,34.054737,0,0,1161
329,bfb2c8d8-7733-4f99-b236-a980418ac0c9,KEN_JrokezFTW,1,0,0,6,47.29146,0,5,0,0,0.000000,1,0,1161
330,bfb2c8d8-7733-4f99-b236-a980418ac0c9,10K_Grown,1,1,0,1,192.77762,0,0,1,1,284.619170,0,0,1161


In [5]:
print(f'{tournament_index_id}_match_info')

eu-trt22_match_info


In [ ]:
# DB와 connect
# create_engine("mysql+pymysql://아이디:"+"암호"+"@주소:포트/데이터베이스이름?charset=utf8", encoding='utf-8')
engine = create_engine("mysql+pymysql://admin:"+password+"@"+endpoint+":3306/"+dbname+"?charset=utf8", encoding="utf-8")
conn_create = engine.connect()

# AWS RDS에 tournament_info 테이블 create
tournament_info.to_sql(name='tournament_info', con=conn_create, if_exists='replace', index=False)
cur_match_info.to_sql(name=f'{tournament_index_id}_match_info', con=conn_create, if_exists='replace', index=False)
conn_create.close()

# DB connect
conn = pymysql.connect(host=endpoint, user=username, password=password, db=dbname, charset='utf8', port=3306)
cur = conn.cursor()

# 업로드된 테이블 확인(SQL 사용)
sql = "SHOW TABLES"
cur.execute(sql)
result = cur.fetchall()
print("\n\nSHOW TABLES: \n",result,"\n\n")

# 내용 확인(SELECT *)
sql = "SELECT * FROM tournament_info"
cur.execute(sql)
result1 = cur.fetchall()
result1 = pd.DataFrame(result1)
print("\ntournament_info(database):\n",result1)